In [ ]:
import numpy as np
import pandas as pd
import squidpy as sq
from cellrefiner import cellrefiner
import seaborn as sns
import math
import matplotlib.pyplot as plt
from scipy.spatial import distance_matrix
import scanpy as sc
from sklearn.neighbors import kneighbors_graph
from scipy.stats import pearsonr
import squidpy as sq
from scipy.sparse import csr_matrix
from scipy.spatial.distance import cdist
from scipy.optimize import linear_sum_assignment
from scipy.spatial.distance import cdist
from scipy.stats import gaussian_kde
from scipy.special import kl_div
import random
from sklearn.neighbors import NearestNeighbors
import scipy
import ot
import tangram as tg
from anndata import AnnData

Fig 2

In [ ]:
# import Bregma mouse data from squidpy MERFISH dataset
adata = sq.datasets.merfish()
adata_slice = adata[adata.obs.Bregma == -9].copy()
adata_slice=adata_slice[adata_slice.obsm['spatial'][:,1]>0.2,:]
ind=random.sample(range(adata_slice.shape[0]),(32*32*5))
adata_slice=adata_slice[ind,:]
coordinates=adata_slice.obsm['spatial'].toarray()
gene_x=adata_slice.X.toarray()
xc=coordinates*5000 
cell_types=adata_slice.obs['Cell_class']
cell_types_num=pd.Categorical(cell_types).codes

x = np.linspace(np.min(xc,axis=0)[0]+100, np.max(xc,axis=0)[0]-100, 32)
y = np.linspace(np.min(xc,axis=0)[1]+100, np.max(xc,axis=0)[1]-100, 32)
xx, yy = np.meshgrid(x, y)
a=np.tile(xx.flatten(),(5,1)).T.flatten()
b=np.tile(yy.flatten(),(5,1)).T.flatten()
xs=np.concatenate(([a],[b]),axis=0).T

neigh = NearestNeighbors(n_neighbors=5)
neigh.fit(xc)
x_id=neigh.kneighbors(xs)

x_idd=x_id[1][0::5,:].flatten()
xc0=xc[x_idd,:]
X_sc2m1=gene_x[x_idd,:]
X_sc_mean=np.zeros((int(x_id[1].shape[0]/5),gene_x.shape[1]))
for i in range(X_sc_mean.shape[0]):
    X_sc_mean[i,:]=np.sum(gene_x[x_id[1][i*5,:],:],axis=0)

x = np.linspace(np.min(xc,axis=0)[0]+100, np.max(xc,axis=0)[0]-100, 32)
y = np.linspace(np.min(xc,axis=0)[1]+100, np.max(xc,axis=0)[1]-100, 32)
xx, yy = np.meshgrid(x, y)
xsm=np.concatenate(([xx.flatten()],[yy.flatten()]),axis=0).T 

In [ ]:
adata_sc=AnnData(gene_x)
adata_sc.obsm['spatial']=xc
adata_st=AnnData(X_sc_mean)
adata_st.obsm['spatial']=xsm

In [ ]:
xc1=xs+np.random.normal(0,150,size=xs.shape)

In [ ]:
sc.pp.pca(adata_sc)
X_sc2m2=adata_sc.obsm['X_pca']
x_id1=[] 
for i in range(xs.shape[0]):
    x_id1.append(np.linalg.norm(xs-xs[i,:],axis=1)<150) 

In [ ]:
def V_xy(xi,xj,V0,U0,xi1,xi2):
    r2=(xj[0]-xi[0])**2+(xj[1]-xi[1])**2
    r=np.sqrt(r2)
    dVdr=-2*r*V0/xi1**2*np.exp(-r2/xi1**2)+2*r/xi2**2*U0*np.exp(-r2/xi2**2)
    drdx=(xj[0]-xi[0])*r**(-0.5)
    drdy=(xj[1]-xi[1])*r**(-0.5)
    dVdx=dVdr*drdx
    dVdy=dVdr*drdy
    
    return np.array([dVdx,dVdy])

In [ ]:
def F_gc(xi,xj,gi,gj): 
    c=pearsonr(gi,gj)[0]
    if np.logical_and(np.linalg.norm(xi-xj)>0,c>0):
        f=c*(xi-xj)/np.linalg.norm(xi-xj)
    else:
        f=0
    return f

In [ ]:
def F_spot(xi,si,rS):
    var1=si-xi
    f=var1/np.reshape(np.linalg.norm(var1,axis=1),(len(np.linalg.norm(var1,axis=1)),1))
    fs=np.minimum((np.linalg.norm(var1,axis=1)-rS)**2,30)
    f[np.linalg.norm(var1,axis=1)<rS]=0
    return f*np.reshape(fs,(len(fs),1))

In [ ]:
db = pd.read_csv(R'LR.csv')
tl = np.zeros((adata.X.shape[0], len(db['interaction_name'])))  
tr = np.zeros((adata.X.shape[0], len(db['interaction_name'])))
for i in range(len(db['interaction_name'])):
    int_name = db['interaction_name'][i].split('_')  
    lig = int_name[0]  
    rec = int_name[1:]  
    lig_ind = adata_sc.var_names == lig  
    if sum(lig_ind) > 0:
        tl[:, i] = adata.X[:, lig_ind].flatten()
        check = 0
        for j in range(len(rec)):  
            if sum(adata_sc.var_names == rec[j]) > 0:
                check += 1

        if check == len(rec):
                rec_ct = adata.X[:, adata_sc.var_names == rec[0]]
                for j in range(len(rec)):
                    rec_ct = np.minimum(rec_ct, a[:, adata_sc.var_names == rec[j]])

                tr[:, i] = rec_ct.flatten()

    W = np.add(np.matmul(tl, tr.T), np.matmul(tr, tl.T))  
    W = np.divide(W, np.amax(W))

In [ ]:
def pre_cal1(N):
        degree=np.diag(np.sum(N,axis=1))
        L=degree-N
        L_inv=np.linalg.pinv(L,rcond=np.finfo(float).tiny)
        n=len(N)
        q=np.multiply(N,np.diag(L_inv)+np.reshape(np.diag(L_inv.T),(-1,1))-L_inv.T-L_inv)
        return q

In [ ]:
def sparsify(W1,q):
        n=len(W1)
        P=np.minimum(1,np.log(n)*q)
        H=np.ones((n,n))*np.finfo(float).tiny
        rand=np.random.rand(n,n)
        idx=np.where(rand<P)
        H[idx]=W1[idx]/P[idx]
        kept=len(H[H>np.finfo(float).tiny])/n**2
        percent=.40
        time=0
        while kept<percent:
            if time>300:
                break
            rand=np.random.rand(n,n)
            idx=np.where(rand<P)
            H[idx]=W1[idx]/P[idx]
            kept=len(H[H>np.finfo(float).tiny])/n**2
            time+=1
            print('kept is:',kept)

        H=H/np.amax(H)
        return H

In [ ]:
m_val = 125  # scaling param
U0 = 0.1 / (2.85 / m_val)
V0 = 1.1 / (2.85 / m_val)
xi1 = 1.21 / (2.85 / m_val)
xi2 = 1.9 / (2.85 / m_val)

degree = np.diag(np.sum(W, axis=1))
L = degree - W
if np.linalg.det(L) > 0:
    q = pre_cal1(W)
    H = sparsify(W, q)
else:
    H = np.zeros(np.shape(W))

pos_s = np.tile(xs, [iterations + 1, 1, 1])
pos = np.tile(xc, [iterations + 1, 1, 1])  
F_gc_const = np.linspace(1, 0, iterations) ** 2
for i in range(iterations):
    p = pos[i, :, :].copy()
    p += F_spot(pos[i, :, :], pos_s[i, :, :], rS*.7)
    for j in range(pos.shape[1]):
        for k in np.arange(0, xc.shape[0])[x_id1[j]]:
            if j != k:
                dv = V_xy(pos[i, k, :], pos[i, j, :], V0, U0, xi1, xi2)
                p[j, :] += -dt * 7100 * dv
                p[j, :] += 4 * F_gc_const[i] * F_gc(pos[i, k, :], pos[i, j, :], X_sc2m2[k, :],X_sc2m2[j, :])  
                p[j, :] += 4 * F_gc_const[i] * H[k, j]
    pos[i + 1, :, :] = p
    
    for j in range(len(cell_num)):
        kernel1=gaussian_kde(pos[i,cell_types_num[x_idd]==cell_num[j],:].T,bw_method=.1) # kernel for output
        kernel2=gaussian_kde(xc[cell_types_num==cell_num[j],:].T,bw_method=.1) # kernel for original distribution
        X, Y = np.mgrid[xs.min(axis=0)[0]:xs.max(axis=0)[0]:10j, xs.min(axis=0)[1]:xs.max(axis=0)[1]:10j] # grid for sampling
        positions = np.vstack([X.ravel(), Y.ravel()])
        err[j,i]=sum(kl_div(kernel1(positions),kernel2(positions)+10e-10))

In [ ]:
plt.scatter(pos[-1,:,0],pos[-1,:,1],c=cell_types_num[x_idd]==5)# output for cell type 5
plt.gca().set_aspect('equal')

In [ ]:
plt.scatter(X,Y,c=kernel2(positions)) # kernel

Fig 3

In [ ]:
ad_map = tg.map_cells_to_space(adata_sc, adata_st,
    mode="cells",
    density_prior='rna_count_based',
    num_epochs=500,
    #device="cuda:0",
    device='cpu',
)

In [ ]:
xc1=xs+np.random.normal(0,75,size=xs.shape)
gmap=ad_map.X
cell5=np.zeros((gmap.shape[1],5))
gmap1=gmap
for i in range(gmap1.shape[1]):
    cell5[i,:]= np.argpartition(gmap1[:,i], -5)[-5:]
    gmap1[cell5[i,:].astype(int),:]=0
    
cell5m=cell5.flatten().astype(int)

In [ ]:
adata_sc=adata_sc[cell5m,:]
X_sc2m2=adata_sc.X.toarray()

In [ ]:
tl = np.zeros((adata.X.shape[0], len(db['interaction_name'])))  
tr = np.zeros((adata.X.shape[0], len(db['interaction_name'])))
for i in range(len(db['interaction_name'])):
    int_name = db['interaction_name'][i].split('_')  
    lig = int_name[0]  
    rec = int_name[1:]  
    lig_ind = adata_sc.var_names == lig  
    if sum(lig_ind) > 0:
        tl[:, i] = adata.X[:, lig_ind].flatten()
        check = 0
        for j in range(len(rec)):  
            if sum(adata_sc.var_names == rec[j]) > 0:
                check += 1

        if check == len(rec):
                rec_ct = adata.X[:, adata_sc.var_names == rec[0]]
                for j in range(len(rec)):
                    rec_ct = np.minimum(rec_ct, a[:, adata_sc.var_names == rec[j]])

                tr[:, i] = rec_ct.flatten()

    W = np.add(np.matmul(tl, tr.T), np.matmul(tr, tl.T))  
    W = np.divide(W, np.amax(W))

In [ ]:
m_val = 125  # scaling param
U0 = 0.1 / (2.85 / m_val)
V0 = 1.1 / (2.85 / m_val)
xi1 = 1.21 / (2.85 / m_val)
xi2 = 1.9 / (2.85 / m_val)

degree = np.diag(np.sum(W, axis=1))
L = degree - W
if np.linalg.det(L) > 0:
    q = pre_cal1(W)
    H = sparsify(W, q)
else:
    H = np.zeros(np.shape(W))

pos_s = np.tile(xs, [iterations + 1, 1, 1])
pos = np.tile(xc, [iterations + 1, 1, 1])  
F_gc_const = np.linspace(1, 0, iterations) ** 2
for i in range(iterations):
    p = pos[i, :, :].copy()
    p += F_spot(pos[i, :, :], pos_s[i, :, :], rS*.7)
    for j in range(pos.shape[1]):
        for k in np.arange(0, xc.shape[0])[x_id1[j]]:
            if j != k:
                dv = V_xy(pos[i, k, :], pos[i, j, :], V0, U0, xi1, xi2)
                p[j, :] += -dt * 7100 * dv
                p[j, :] += 4 * F_gc_const[i] * F_gc(pos[i, k, :], pos[i, j, :], X_sc2m2[k, :],X_sc2m2[j, :])  
                p[j, :] += 4 * F_gc_const[i] * H[k, j]
    pos[i + 1, :, :] = p
    
    for j in range(len(cell_num)):
        kernel1=gaussian_kde(pos[i,cell_types_num[x_idd]==cell_num[j],:].T,bw_method=.1) # kernel for output
        kernel2=gaussian_kde(xc[cell_types_num==cell_num[j],:].T,bw_method=.1) # kernel for original distribution
        X, Y = np.mgrid[xs.min(axis=0)[0]:xs.max(axis=0)[0]:10j, xs.min(axis=0)[1]:xs.max(axis=0)[1]:10j] # grid for sampling
        positions = np.vstack([X.ravel(), Y.ravel()])
        err[j,i]=sum(kl_div(kernel1(positions),kernel2(positions)+10e-10))

In [ ]:
adata_sc1=AnnData(gene_x)
adata_sc1.obsm['spatial']=pos[-1,:,:]
adata_sc1.obsm['spatial'][:,1]=-adata_sc1.obsm['spatial'][:,1]
adata_sc1.obs['Cell_class']=cell_types_num[x_idd]

In [ ]:
x=adata_sc1.obsm['spatial']
a=adata_sc1.obs['Cell_class']
cc=np.zeros((len(np.unique(a)),len(np.unique(a))))
d1=d<500
for i in range(len(np.unique(a))):
    for j in range(len(np.unique(a))):
        temp=d1[a==i,:]
        cc[i,j]=np.sum(temp[:,a==j])

In [ ]:
# cc contact map
sns.heatmap(cc/np.max(cc,axis=0),yticklabels=np.unique(adata_sc1.obs['Cell_class']))

In [ ]:
# seqfish
adata=sq.datasets.seqfish()

In [ ]:
ex=np.random.choice(adata.shape[0],size=2000,replace=False)
adata_st=adata[ex,:].copy()
sc_d=np.setdiff1d(np.arange(0,adata.shape[0]),ex)
adata_sc=adata[np.random.choice(sc_d.shape[0],size=10000,replace=False),:].copy()

In [ ]:
neigh = NearestNeighbors(n_neighbors=5)
neigh.fit(adata_sc.obsm['spatial'])
x_id=neigh.kneighbors(adata_st.obsm['spatial'])
gene_x=adata_sc.X.toarray()
X_sc_mean=np.zeros((adata_st.X.shape))
for i in range(X_sc_mean.shape[0]):
    X_sc_mean[i,:]=np.sum(gene_x[x_id[1][i],:],axis=0)

In [ ]:
adata_sc.obs['Cell_class']=pd.Categorical(adata_sc.obs['label'])
adata_st.X=X_sc_mean

In [ ]:
adata_sc.obs['Cell_class']=pd.Categorical(adata_sc.obs['celltype_mapped_refined'])
adata_st.X=X_sc_mean

In [ ]:
cr = cellrefiner.CellRefiner(adata_st=adata_st,adata_sc=adata_sc,db=db)
cr.gen_w()
cr.pp_cr(group = "Cell_class")
cr.sim_cr(iterations=8,W=cr.W,tissue_bound=1)

In [ ]:
ct_c=cr.pos[0,:,:]

e=0
for i in np.unique(cell_codes): # loop over cell types
    xs=adata_sc.obsm['spatial'][pd.Categorical(adata_sc.obs['Cell_class']).codes==i,:] # spatial coords for ground truth
    xt=pos[0,cell_codes==i,:] # spatial coords for simulation
    a = np.array(np.ones((xs.shape[0],)) / xs.shape[0]) # uniform distribution on samples
    b = np.array(np.ones((xt.shape[0],)) / xt.shape[0] ) # uniform distribution on samples
    M = ot.dist(xs, xt,metric='euclidean') 
    M=M/np.max(M)
    e+=ot.bregman.sinkhorn2(a,b,M,1)
    
print(e)

x,y=np.meshgrid(np.linspace(np.amin(ct_c[:,0]),np.amax(ct_c[:,0]),100),np.linspace(np.amin(ct_c[:,1]),np.amax(ct_c[:,1]),100))
positions = np.vstack([x.ravel(), y.ravel()])
e1=0
for i in np.unique(cell_codes): # loop over cell types
    xs=adata_sc.obsm['spatial'][pd.Categorical(adata_sc.obs['Cell_class']).codes==i,:] # spatial coords for ground truth
    xt=pos[0,cell_codes==i,:] # spatial coords for simulation
    kernel1=gaussian_kde(xs.T,.1)
    kernel2=gaussian_kde(xt.T,.1)
    e1+=sum(kl_div(kernel1(positions),kernel2(positions)))
    
print(e1)

In [ ]:
x=adata_sc.obsm['spatial']
a=adata_sc.obs['Cell_class']
cc=np.zeros((len(np.unique(a)),len(np.unique(a))))
d1=d<500
for i in range(len(np.unique(a))):
    for j in range(len(np.unique(a))):
        temp=d1[a==i,:]
        cc[i,j]=np.sum(temp[:,a==j])

In [ ]:
sns.heatmap(cc/np.max(cc,axis=0),yticklabels=np.unique(adata_sc.obs['Cell_class']))
sns.heatmap(cc/np.max(cc,axis=0),yticklabels=np.unique(adata_sc.obs['Cell_class']))

In [ ]:
# starmap

In [ ]:
adata=sc.read_h5ad(R"starmap.h5ad")

In [ ]:
ex=np.random.choice(adata.shape[0],size=200,replace=False)
adata_st=adata[ex,:].copy()
sc_d=np.setdiff1d(np.arange(0,adata.shape[0]),ex)
adata_sc=adata[np.random.choice(sc_d.shape[0],size=1000,replace=False),:].copy()

In [ ]:
neigh = NearestNeighbors(n_neighbors=5)
neigh.fit(adata_sc.obsm['spatial'])
x_id=neigh.kneighbors(adata_st.obsm['spatial']
gene_x=adata_sc.X

X_sc_mean=np.zeros((adata_st.X.shape))
for i in range(X_sc_mean.shape[0]):
    X_sc_mean[i,:]=np.sum(gene_x[x_id[1][i],:],axis=0)

In [ ]:
adata_sc.obs['Cell_class']=pd.Categorical(adata_sc.obs['label'])
adata_st.X=X_sc_mean

In [ ]:
cr = cellrefiner.CellRefiner(adata_st=adata_st,adata_sc=adata_sc,db=db)
cr.gen_w()
cr.pp_cr(group = "Cell_class")
cr.sim_cr(iterations=10,W=cr.W,tissue_bound=1)

In [ ]:
#slideseq

In [ ]:
adata = sq.datasets.slideseqv2()
ex=np.random.choice(adata.shape[0],size=2000,replace=False)
adata_st=adata[ex,:].copy()
sc_d=np.setdiff1d(np.arange(0,adata.shape[0]),ex)
adata_sc=adata[np.random.choice(sc_d.shape[0],size=10000,replace=False),:].copy()
neigh = NearestNeighbors(n_neighbors=5)
neigh.fit(adata_sc.obsm['spatial'])
x_id=neigh.kneighbors(adata_st.obsm['spatial']
gene_x=adata_sc.X

X_sc_mean=np.zeros((adata_st.X.shape))
for i in range(X_sc_mean.shape[0]):
    X_sc_mean[i,:]=np.sum(gene_x[x_id[1][i],:],axis=0)
                      
adata_sc.obs['Cell_class']=pd.Categorical(adata_sc.obs['cluster'])
adata_st.X=X_sc_mean

In [ ]:
cr = cellrefiner.CellRefiner(adata_st=adata_st,adata_sc=adata_sc,db=db)
cr.gen_w()
cr.pp_cr(group = "Cell_class")
cr.sim_cr(iterations=10,W=cr.W,tissue_bound=1)

In [ ]:
# visium 

In [ ]:
adata_st = sq.datasets.visium_fluo_adata_crop()
adata_st = adata_st[
    adata_st.obs.cluster.isin([f"Cortex_{i}" for i in np.arange(1, 5)])
].copy()
img = sq.datasets.visium_fluo_image_crop()

adata_sc = sq.datasets.sc_mouse_cortex()

In [ ]:
overlapping_genes=list(set(adata_st.var_names)&set(adata_sc.var_names)) # subset to overlapping genes
adata_st=adata_st[:,overlapping_genes].copy()
adata_sc=adata_sc[:,overlapping_genes].copy()

In [ ]:
cr = cellrefiner.CellRefiner(adata_st=adata_st,adata_sc=adata_sc,db=db)
cr.gen_w()
cr.pp_cr(group = "cell_subclass")
cr.sim_cr(iterations=10,W=cr.W,tissue_bound=1)

In [ ]:
# ccc

In [ ]:
L_id=list(np.nonzero(adata_sc.var_names=='Cd46'))[0][0]
R_id=list(np.nonzero(adata_sc.var_names=='Jag1'))[0][0]
L=adata_sc.X[cr.cell5m,L_id]
R=adata_sc.X[cr.cell5m,R_id]
P=np.outer(L,R)/(0.5+np.outer(L,R))

d=distance_matrix(cr.pos[-1,:,:],cr.pos[-1,:,:])
d_mask=d<250
c1=P*d_mask
c=cell_types_num

t1=np.zeros((adata_sc.shape[0],len(np.unique(adata_sc.obs['cell_subclass'])))) 
for i in range(adata_sc.shape[0]):
    t1[i,c[i]]=1
cst_num=np.sum(t1,axis=0)     
txct_st=np.matmul(np.matmul(np.transpose(t1),c1),t1)
txct=np.divide(txct_st,np.outer(cst_num,cst_num)) 

cst_labels=np.unique(pd.Categorical(cell_types))
sns.heatmap(txct_st,xticklabels=cst_labels,yticklabels=cst_labels)

In [ ]:
import scanorama
ad1=[adata_sc,adata_st] # adatas_anterior
ad2=scanorama.correct_scanpy(ad1,return_dimred=True) # adatas_cor_anterior

In [ ]:
dists=1-cosine_distances(ad2[0][0],ad2[0][1])

In [ ]:
def label_transfer(dist, labels):
    lab = pd.get_dummies(labels).to_numpy().T
    class_prob = lab @ dist
    norm = np.linalg.norm(class_prob, 2, axis=0)
    class_prob = class_prob / norm
    class_prob = (class_prob.T - class_prob.min(1)) / class_prob.ptp(1)
    return class_prob

In [ ]:
class_prob = label_transfer(dists, adata_sc.obs.cell_subclass)

In [ ]:
cst_num=np.sum(class_prob,axis=0)

In [ ]:
txct_st=np.matmul(np.matmul(np.transpose(class_prob),c1),class_prob)
txct=np.divide(txct_st,np.outer(cst_num,cst_num))

In [ ]:
txct_st=np.matmul(np.matmul(np.transpose(class_prob),c1),class_prob)

In [ ]:
cst_labels=np.unique(adata_sc.obs.cell_subclass)
sns.heatmap(txct_st,xticklabels=cst_labels,yticklabels=cst_labels)